In [1]:
import tensorflow as tf
import numpy as np
import os
import glob
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from PIL import Image, ImageOps
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt
from rnnconv import RnnConv

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16174578289840465323
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17346590504406897574
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9994237376
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15102584035585668186
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:22:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2020297650506535570
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
print(tf.config.list_physical_devices("GPU"))
iterations = 1

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images2, train_labels2), (test_images2, test_labels2) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images2, test_iamges2 = train_images2 / 255.0, test_images / 255.0

train_images = np.concatenate((train_images, train_images2) ,axis=0)
test_images = np.concatenate((test_images, test_images2), axis=0)

train_labels = np.concatenate((train_labels, train_labels2), axis=0)
test_labels = np.concatenate((test_labels, test_labels2), axis=0)

In [6]:
with open('D:\\\data\\data.npy', 'rb') as f:
    data = (np.load(f) / 255.0).reshape((99999,32,32,3))

In [7]:
with open('D:\\\data\\data2.npy', 'rb') as f:
    data2 = (np.load(f) / 255.0)
    num_images = data2.shape[0] // (32*32*3)
    
    data2 = np.rot90(data2.reshape((num_images,32,32,3)), axes=[1,2])

train_images = np.concatenate((train_images, data), axis=0)
train_images = np.concatenate((train_images, data2), axis=0)

del data2
del data

rotated_images = np.rot90(train_images, axes=[1,2])

train_images = np.concatenate((train_images, rotated_images), axis=0)
del rotated_images

In [5]:
print("There are {} images".format(train_images.shape[0]))

There are 100000 images


In [4]:
model = tf.keras.Sequential() 
relu_amount = 0.01

for i in range(iterations):

    model.add(layers.Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), strides = (2, 2), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Conv2D(filters=32, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

    model.add(layers.Conv2D(filters=512, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=2, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))
    
    
    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
    model.add(layers.ConvLSTM2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))
    model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

    model.add(layers.Conv2D(filters=3, kernel_size=1, strides = (1,1), padding='same', data_format='channels_last'))
    model.add(LeakyReLU(alpha=relu_amount))

In [5]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, epsilon=1e-06, amsgrad=True)

def SSIMLoss(y_true, y_pred):
    return tf.math.square(1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0)))

model.compile(optimizer=opt, loss=SSIMLoss, metrics=[SSIMLoss])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 64)        0         
_________________________________________________________________
lambda (Lambda)              (None, 1, 16, 16, 64)     0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 8, 8, 256)         2950144   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1, 8, 8, 256)      0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 4, 4, 512)         1

In [8]:
batch_size = 512
train_data_url = "D:\\data"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        rotation_range=90,
        validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_url,
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='input',
        color_mode='rgb',
        classes=None)

model.fit(train_generator, epochs=5)

Found 195556 images belonging to 1 classes.
Epoch 1/5
382/382 [==============================] - 6258s 16s/step - loss: 0.3816 - SSIMLoss: 0.3816
Epoch 2/5
382/382 [==============================] - 5725s 15s/step - loss: 0.2135 - SSIMLoss: 0.2135
Epoch 3/5
382/382 [==============================] - 5743s 15s/step - loss: 0.1561 - SSIMLoss: 0.1561
Epoch 4/5
382/382 [==============================] - 5753s 15s/step - loss: 0.1269 - SSIMLoss: 0.1269
Epoch 5/5
382/382 [==============================] - 5735s 15s/step - loss: 0.1070 - SSIMLoss: 0.1070


In [8]:
model3 = model.fit(train_images, train_images, batch_size=32, epochs=5, validation_split=0.05)

Epoch 1/20
 390/2969 [==>...........................] - ETA: 5:49 - loss: 0.3852 - SSIMLoss: 0.3852

KeyboardInterrupt: 

In [9]:
from matplotlib import pyplot as plt

img = train_images_[400]
img_ = img.numpy().reshape((32,32,3))
plt.imshow(img_, interpolation='nearest')
print(img_.shape)
plt.show()

NameError: name 'train_images_' is not defined

In [10]:
# img_pred = test_images[0:10].reshape((10,3,32,32,))
full = model.predict(test_images[20:30,:,:,:])

NameError: name 'test_images' is not defined

In [ ]:
plt.plot(model3.history["val_SSIMLoss"])

In [ ]:
# img_pred_full= img_pred.reshape((10,32,32,3))

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[20:30,:,:,:][i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
plt.show()

In [ ]:
full_pred = full.reshape((10,32,32,3))

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(full_pred[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
plt.show()

### Loading Data

In [ ]:
data_url = "D:\\data\\train_f"

class ImageLoader:
    
    def __init__(self, image_directory):
        
        self.image_directory = image_directory
        self.image_paths = [os.path.abspath(os.path.join(self.image_directory, p)) for p in os.listdir(self.image_directory) if p.endswith(('jpg', 'png'))]
        
        print("There are {} iamges".format(len(self.image_paths)))
        
        self.num_images = len(self.image_paths)
        self.garbage_counter = 0
        self.process_images(self.num_images)
        
    def process_images(self, num_images):
        
        self.data = np.array([])
        
        
        for i in range(num_images):
            
            if i % 10000 == 0:
                print("Processed image {}".format(i))
            
            image_filename = "image%s.png" % i
            
            PIL_image = Image.open(self.image_paths[i]).resize((32, 32), Image.ANTIALIAS)
             
            image_as_array = np.array(PIL_image)

            if image_as_array.shape != (32,32,3):
                self.garbage_counter += 1
                continue
    
            self.data = np.append(self.data, image_as_array)

loader = ImageLoader(data_url)
data = loader.data

with open('D:\\data\\data2.npy', 'wb') as f:
    np.save(f, data)